# Geração de Dataset Sintético DNE

Este notebook gera ~10k endereços brasileiros sintéticos com:
- 85% registros limpos
- 15% com variações (bairro vazio, abreviações, typos)
- CEPs geográficos reais por estado
- Queries de teste categorizadas

In [1]:
import pandas as pd
import numpy as np
import random
from pathlib import Path

In [2]:
# Seed para reprodutibilidade
random.seed(42)
np.random.seed(42)

## Dados Base - Estados e Cidades Brasileiras

In [4]:
# Estados com faixas de CEP reais
STATES = {
    'SP': {'cep_range': (1000, 19999), 'cities': ['São Paulo', 'Campinas', 'Santos', 'Ribeirão Preto', 'Sorocaba', 'São José dos Campos', 'Osasco', 'Santo André']},
    'RJ': {'cep_range': (20000, 28999), 'cities': ['Rio de Janeiro', 'Niterói', 'São Gonçalo', 'Duque de Caxias', 'Nova Iguaçu', 'Campos dos Goytacazes', 'Petrópolis']},
    'MG': {'cep_range': (30000, 39999), 'cities': ['Belo Horizonte', 'Uberlândia', 'Contagem', 'Juiz de Fora', 'Betim', 'Montes Claros', 'Uberaba']},
    'BA': {'cep_range': (40000, 48999), 'cities': ['Salvador', 'Feira de Santana', 'Vitória da Conquista', 'Camaçari', 'Itabuna', 'Juazeiro', 'Lauro de Freitas']},
    'PR': {'cep_range': (80000, 87999), 'cities': ['Curitiba', 'Londrina', 'Maringá', 'Ponta Grossa', 'Cascavel', 'São José dos Pinhais', 'Foz do Iguaçu']},
    'RS': {'cep_range': (90000, 99999), 'cities': ['Porto Alegre', 'Caxias do Sul', 'Pelotas', 'Canoas', 'Santa Maria', 'Gravataí', 'Novo Hamburgo']},
    'PE': {'cep_range': (50000, 56999), 'cities': ['Recife', 'Jaboatão dos Guararapes', 'Olinda', 'Caruaru', 'Petrolina', 'Paulista', 'Cabo de Santo Agostinho']},
    'CE': {'cep_range': (60000, 63999), 'cities': ['Fortaleza', 'Caucaia', 'Juazeiro do Norte', 'Maracanaú', 'Sobral', 'Crato', 'Itapipoca']},
    'SC': {'cep_range': (88000, 89999), 'cities': ['Florianópolis', 'Joinville', 'Blumenau', 'São José', 'Criciúma', 'Chapecó', 'Itajaí']},
    'GO': {'cep_range': (72800, 76799), 'cities': ['Goiânia', 'Aparecida de Goiânia', 'Anápolis', 'Rio Verde', 'Luziânia', 'Águas Lindas de Goiás']}
}

# Prefixos de logradouro
STREET_TYPES = [
    'Rua', 'Avenida', 'Travessa', 'Alameda', 'Praça', 'Rodovia',
    'Estrada', 'Viela', 'Largo', 'Beco'
]

# Nomes de ruas comuns
STREET_NAMES = [
    'das Flores', 'do Comércio', 'Principal', 'Central', 'São João', 'São José',
    'Sete de Setembro', 'Quinze de Novembro', 'Tiradentes', 'Dom Pedro II',
    'Barão do Rio Branco', 'Getúlio Vargas', 'Santos Dumont', 'Presidente Vargas',
    'das Acácias', 'dos Pinheiros', 'das Palmeiras', 'dos Ipês', 'das Hortênsias',
    'do Sol', 'da Paz', 'da Liberdade', 'da Independência', 'da República',
    'A', 'B', 'C', 'D', 'Primeiro', 'Segundo', 'Terceiro', 'Quarto',
    'Amazonas', 'Paraná', 'Tocantins', 'Araguaia', 'São Francisco',
    'José da Silva', 'Maria Santos', 'João Pereira', 'Antonio Costa',
    'Pedro Álvares Cabral', 'Duque de Caxias', 'Marechal Deodoro',
    'das Américas', 'do Atlântico', 'Paulista', 'Copacabana', 'Ipanema'
]

# Padrões de bairro
NEIGHBORHOOD_PREFIXES = ['', 'Jardim', 'Vila', 'Parque', 'Conjunto', 'Cidade']
NEIGHBORHOOD_NAMES = [
    'Centro', 'Primavera', 'Esperança', 'Nova', 'Velha', 'Industrial',
    'São Pedro', 'Santa Maria', 'São Francisco', 'Santo Antonio',
    'das Flores', 'dos Pássaros', 'das Árvores', 'do Sol', 'da Lua',
    'Alto', 'Baixo', 'Norte', 'Sul', 'Leste', 'Oeste',
    'Europa', 'América', 'Brasil', 'Planalto', 'Serra'
]

## Funções Auxiliares

In [5]:
def generate_cep(state_code: str, street_name: str) -> str:
    """Gera CEP baseado no estado com variação por rua"""
    cep_min, cep_max = STATES[state_code]['cep_range']
    
    # Hash do nome da rua para consistência (mesma rua = CEP similar)
    street_hash = hash(street_name) % 1000
    base_cep = cep_min + street_hash
    
    # Adiciona variação aleatória pequena
    cep = base_cep + random.randint(0, 50)
    cep = min(cep, cep_max)
    
    # Formata: XXXXX-XXX
    suffix = random.randint(0, 999)
    return f"{cep:05d}-{suffix:03d}"

def generate_neighborhood() -> str:
    """Gera nome de bairro"""
    prefix = random.choice(NEIGHBORHOOD_PREFIXES)
    name = random.choice(NEIGHBORHOOD_NAMES)
    
    if prefix:
        return f"{prefix} {name}"
    return name

def generate_street() -> str:
    """Gera nome de logradouro"""
    street_type = random.choice(STREET_TYPES)
    street_name = random.choice(STREET_NAMES)
    return f"{street_type} {street_name}"

def apply_abbreviation(text: str) -> str:
    """Aplica abreviações comuns"""
    replacements = {
        'Rua': 'R.',
        'Avenida': 'Av.',
        'Travessa': 'Trav.',
        'Alameda': 'Alam.',
        'Praça': 'Pça.',
        'Jardim': 'Jd.',
        'Vila': 'Vl.',
    }
    
    for full, abbr in replacements.items():
        if text.startswith(full):
            return text.replace(full, abbr, 1)
    return text

def apply_typo(text: str) -> str:
    """Aplica typo leve (troca 1-2 caracteres)"""
    if len(text) < 5:
        return text
    
    text_list = list(text)
    
    # Troca 1 caractere aleatório
    idx = random.randint(2, len(text_list) - 2)
    
    typo_types = ['swap', 'duplicate', 'remove']
    typo_type = random.choice(typo_types)
    
    if typo_type == 'swap' and idx < len(text_list) - 1:
        text_list[idx], text_list[idx + 1] = text_list[idx + 1], text_list[idx]
    elif typo_type == 'duplicate':
        text_list.insert(idx, text_list[idx])
    elif typo_type == 'remove':
        text_list.pop(idx)
    
    return ''.join(text_list)

## Geração do Dataset Principal

In [6]:
def generate_dne_dataset(n_records: int = 10000) -> pd.DataFrame:
    """Gera dataset DNE sintético"""
    
    records = []
    
    # Calcula quantidade de registros por categoria
    n_clean = int(n_records * 0.85)
    n_empty_bairro = int(n_records * 0.05)
    n_abbreviations = int(n_records * 0.05)
    n_typos = int(n_records * 0.05)
    
    categories = (
        ['clean'] * n_clean +
        ['empty_bairro'] * n_empty_bairro +
        ['abbreviation'] * n_abbreviations +
        ['typo'] * n_typos
    )
    
    # Ajusta para garantir total exato
    while len(categories) < n_records:
        categories.append('clean')
    categories = categories[:n_records]
    
    random.shuffle(categories)
    
    for i, category in enumerate(categories):
        # Seleciona estado e cidade
        state_code = random.choice(list(STATES.keys()))
        cidade = random.choice(STATES[state_code]['cities'])
        
        # Gera endereço base
        logradouro = generate_street()
        bairro = generate_neighborhood()
        cep = generate_cep(state_code, logradouro)
        
        # Aplica variações conforme categoria
        if category == 'empty_bairro':
            bairro = ''
        elif category == 'abbreviation':
            logradouro = apply_abbreviation(logradouro)
            if random.random() < 0.5:
                bairro = apply_abbreviation(bairro)
        elif category == 'typo':
            if random.random() < 0.7:
                logradouro = apply_typo(logradouro)
            else:
                bairro = apply_typo(bairro)
        
        record = {
            'logradouro': logradouro,
            'bairro': bairro,
            'cidade': cidade,
            'uf': state_code,
            'cep': cep
        }
        
        records.append(record)
    
    df = pd.DataFrame(records)
    return df

In [7]:
# Gera o dataset
print("Gerando 10.000 endereços sintéticos...")
df_dne = generate_dne_dataset(10000)
print(f"Dataset gerado: {len(df_dne)} registros")
print(f"\nPrimeiros registros:")
df_dne.head(10)

Gerando 10.000 endereços sintéticos...
Dataset gerado: 10000 registros

Primeiros registros:


,logradouro,bairro,cidade,uf,cep
0,Avenida Central,das Árvores,Goiânia,GO,73730-911
1,Travessa Central,Parque Primavera,Petrópolis,RJ,20514-625
2,Avenida Marechal Deodoro,Norte,Recife,PE,50707-301
3,Avenida João Pereira,Parque Santa Maria,Caruaru,PE,50849-806
4,Viela A,Parque Serra,Goiânia,GO,73714-901
5,Avenida da Liberdade,Cidade Europa,Luziânia,GO,73616-026
6,Rodovia José da Silva,Jardim Nova,Sobral,CE,60201-012
7,Avenida José da Silva,Jardim Sul,Águas Lindas de Goiás,GO,73008-652
8,Rodovia C,Parque Brasil,Blumenau,SC,88573-455
9,Trav. Principal,Velha,Uberlândia,MG,30574-598


In [8]:
# Estatísticas do dataset
print("\n=== Estatísticas do Dataset ===")
print(f"Total de registros: {len(df_dne)}")
print(f"\nDistribuição por UF:")
print(df_dne['uf'].value_counts())
print(f"\nRegistros com bairro vazio: {df_dne['bairro'].isna().sum() + (df_dne['bairro'] == '').sum()}")
print(f"\nExemplos de variações:")
print("\nAbreviações:")
print(df_dne[df_dne['logradouro'].str.contains(r'\.')].head(3))
print("\nBairros vazios:")
print(df_dne[df_dne['bairro'] == ''].head(3))


=== Estatísticas do Dataset ===
Total de registros: 10000

Distribuição por UF:
uf
PE    1046
GO    1042
PR    1030
SC    1014
BA    1011
RJ    1002
CE     976
SP     974
MG     960
RS     945
Name: count, dtype: int64

Registros com bairro vazio: 500

Exemplos de variações:

Abreviações:
         logradouro              bairro          cidade  uf        cep
9   Trav. Principal               Velha      Uberlândia  MG  30574-598
12   Alam. Amazonas  Conjunto Primavera  Belo Horizonte  MG  30840-352
24     Trav. Paraná     Conjunto do Sol        Gravataí  RS  90062-066

Bairros vazios:
                 logradouro bairro          cidade  uf        cep
81      Estrada do Comércio         Rio de Janeiro  RJ  20398-696
123        Praça das Flores               Paulista  PE  50364-099
178  Beco Presidente Vargas                 Santos  SP  01676-575


## Geração de Queries de Teste

In [9]:
def generate_test_queries(df: pd.DataFrame, n_queries: int = 150) -> pd.DataFrame:
    """Gera queries de teste categorizadas"""
    
    queries = []
    
    # Seleciona amostras do dataset para base das queries
    sample_indices = random.sample(range(len(df)), min(n_queries, len(df)))
    
    # Distribuição de categorias
    n_cep_wrong = int(n_queries * 0.30)
    n_abbreviations = int(n_queries * 0.40)
    n_typos = int(n_queries * 0.20)
    n_empty_fields = int(n_queries * 0.10)
    
    categories = (
        ['cep_wrong'] * n_cep_wrong +
        ['abbreviation'] * n_abbreviations +
        ['typo'] * n_typos +
        ['empty_fields'] * n_empty_fields
    )
    
    # Ajusta total
    while len(categories) < n_queries:
        categories.append('abbreviation')
    categories = categories[:n_queries]
    random.shuffle(categories)
    
    for idx, category in zip(sample_indices[:len(categories)], categories):
        row = df.iloc[idx]
        
        query = {
            'logradouro': row['logradouro'],
            'bairro': row['bairro'],
            'cidade': row['cidade'],
            'uf': row['uf'],
            'cep': row['cep'],
            'category': category,
            'expected_index': idx
        }
        
        # Aplica modificações por categoria
        if category == 'cep_wrong':
            # CEP de outro estado ou região
            other_state = random.choice([s for s in STATES.keys() if s != row['uf']])
            query['cep'] = generate_cep(other_state, 'fake')
            
        elif category == 'abbreviation':
            query['logradouro'] = apply_abbreviation(row['logradouro'])
            if row['bairro']:
                query['bairro'] = apply_abbreviation(row['bairro'])
                
        elif category == 'typo':
            if random.random() < 0.6:
                query['logradouro'] = apply_typo(row['logradouro'])
            else:
                query['cidade'] = apply_typo(row['cidade'])
                
        elif category == 'empty_fields':
            # Remove 1-2 campos aleatórios
            fields_to_empty = random.sample(['bairro', 'cep'], random.randint(1, 2))
            for field in fields_to_empty:
                query[field] = ''
        
        queries.append(query)
    
    return pd.DataFrame(queries)

In [10]:
# Gera queries de teste
print("Gerando queries de teste...")
df_queries = generate_test_queries(df_dne, n_queries=150)
print(f"\nQueries geradas: {len(df_queries)}")
print(f"\nDistribuição por categoria:")
print(df_queries['category'].value_counts())
print(f"\nExemplos de queries:")
df_queries.head(10)

Gerando queries de teste...

Queries geradas: 150

Distribuição por categoria:
category
abbreviation    60
cep_wrong       45
typo            30
empty_fields    15
Name: count, dtype: int64

Exemplos de queries:


,logradouro,bairro,cidade,uf,cep,category,expected_index
0,Rua São Jsoé,Vila do Sol,Campos dos Goytacazes,RJ,20995-425,typo,3655
1,Rodovia da Independência,Cidade da Lua,Pelotas,RS,90442-935,abbreviation,3768
2,Rua ds Palmeiras,Vila São Francisco,Feira de Santana,BA,40240-065,typo,3711
3,Largo Dom Pedro II,Parque Europa,Aparecida de Goiânia,GO,88490-104,cep_wrong,3806
4,Praça doo Sol,do Sol,Florianópolis,SC,88862-436,typo,5997
5,Largo C,das Árvores,Aparecida de Goiânia,GO,73655-755,abbreviation,6479
6,Av. das Flores,Parque Serra,Montes Claros,MG,31031-079,abbreviation,2734
7,Becod o Comércio,Jardim São Pedro,Petrópolis,RJ,20800-002,typo,6093
8,Estrada Presidente Vargas,Parque do Sol,Foz do Iguaçu,PR,40470-693,cep_wrong,5517
9,Alameda A,,Caxias do Sul,RS,,empty_fields,280


## Salvar Datasets

In [11]:
# Salva dataset principal
output_path = Path('../data')
output_path.mkdir(exist_ok=True)

dne_file = output_path / 'dne_sample.parquet'
df_dne.to_parquet(dne_file, index=False)
print(f"Dataset DNE salvo em: {dne_file}")

# Salva queries de teste
queries_file = output_path / 'test_queries.parquet'
df_queries.to_parquet(queries_file, index=False)
print(f"Queries de teste salvas em: {queries_file}")

# Salva também CSV para inspeção manual
df_dne.head(100).to_csv(output_path / 'dne_sample_preview.csv', index=False)
df_queries.head(50).to_csv(output_path / 'test_queries_preview.csv', index=False)
print("\nArquivos CSV de preview salvos para inspeção manual")

Dataset DNE salvo em: ..\data\dne_sample.parquet
Queries de teste salvas em: ..\data\test_queries.parquet

Arquivos CSV de preview salvos para inspeção manual


In [ ]:
print("\n✅ Dataset sintético gerado com sucesso!")
print(f"\nPróximos passos:")
print("1. Execute o notebook 'busca_vetorial_poc.ipynb' para construir os índices FAISS")
print("2. Teste a busca com as queries geradas")
print("3. Valide a precisão e ajuste os pesos se necessário")